# Use AutoAI and BatchedTreeEnsemble for incremental learning of AutoAI pipelines `ibm-watson-machine-learning`

This notebook contains the steps and code to demonstrate support of AutoAI experiments in Watson Machine Learning service. It introduces commands for data retrieval, training experiments, persisting pipelines, testing pipelines, refining pipelines, and scoring.

Some familiarity with Python is helpful. This notebook uses Python 3.9.


## Learning goals

The learning goals of this notebook are:

-  Work with Watson Machine Learning experiments to train AutoAI models.
-  Compare trained models quality and select the best one for further refinement.
-  Refine the best model and test new variations.
-  Online deployment and score the trained model.


## Contents

This notebook contains the following parts:

1. [Setup](#setup)
2. [Create connection](#connection)
3. [Optimizer definition](#definition)
4. [Experiment Run](#run)
5. [Pipelines comparison and testing](#comparison)
6. [Incremental learning](#incremental)
7. [Deploy and Score](#scoring)
8. [Cleanup](#cleanup)
9. [Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).
-  Create a <a href="https://console.bluemix.net/catalog/infrastructure/cloud-object-storage" target="_blank" rel="noopener no referrer">Cloud Object Storage (COS)</a> instance (a lite plan is offered and information about how to order storage can be found <a href="https://console.bluemix.net/docs/services/cloud-object-storage/basics/order-storage.html#order-storage" target="_blank" rel="noopener no referrer">here</a>). <br/>**Note: When using Watson Studio, you already have a COS instance associated with the project you are running the notebook in.**


### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide Cloud `API key` and `location`.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.


You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve the instance `location`.

```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```


**NOTE:** You can also get a service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, and then copy the created key and paste it in the following cell.


**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PUT_YOUR_KEY_HERE'
location = 'us-south'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` and dependecies
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="https://ibm.github.io/watson-machine-learning-sdk/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs | tail -n 1
!pip install -U scikit-learn==1.0.2 | tail -n 1
!pip install wget | tail -n 1
!pip install matplotlib | tail -n 1 

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

You need to create a space that will be used for your work. If you do not have a space, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click **New Deployment Space**
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press **Create**
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: assign space ID below

In [ ]:
space_id = 'PUT_YOUR_SPACE_ID_HERE'

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in Watson Machine Learning, you need to set the **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

<a id="connection"></a>
## 2. Connections to COS

Define connection information to COS bucket and training data CSV file. This example uses the German Credit Risk dataset. 

The dataset can be downloaded from [here](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/new/electricity.csv).

**Action**: Upload training data to COS bucket and enter location information below.

In [ ]:
cos_credentials = client.spaces.get_details(space_id=space_id)['entity']['storage']['properties']

filename = 'electricity.csv'
datasource_name = 'bluemixcloudobjectstorage'
bucketname = cos_credentials['bucket_name']

Download training data from git repository.

In [ ]:
import wget
import os

url = "https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/data/electricity/electricity.csv"
if not os.path.isfile(filename): 
    wget.download(url)

#### Create a connection

In [ ]:
conn_meta_props= {
    client.connections.ConfigurationMetaNames.NAME: f"Connection to Database - {datasource_name} ",
    client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: client.connections.get_datasource_type_uid_by_name(datasource_name),
    client.connections.ConfigurationMetaNames.DESCRIPTION: "Connection to external Database",
    client.connections.ConfigurationMetaNames.PROPERTIES: {
        'bucket': bucketname,
        'access_key': cos_credentials['credentials']['editor']['access_key_id'],
        'secret_key': cos_credentials['credentials']['editor']['secret_access_key'],
        'iam_url': 'https://iam.cloud.ibm.com/identity/token',
        'url': cos_credentials['endpoint_url']
    }
}

conn_details = client.connections.create(meta_props=conn_meta_props)

In [ ]:
from ibm_watson_machine_learning.helpers import DataConnection, S3Location

connection_id = client.connections.get_uid(conn_details)

electricity_conn = DataConnection(
    connection_asset_id=connection_id,
    location=S3Location(bucket=bucketname,
                        path=filename))

electricity_conn.set_client(client)
training_data_reference=[electricity_conn]
credit_risk_conn.write(data=filename, remote_name=filename)

<a id="definition"></a>
## 3. Optimizer definition

### Optimizer configuration

Provide the input information for AutoAI optimizer:
- `name` - experiment name
- `prediction_type` - type of the problem
- `prediction_column` - target column name
- `scoring` - optimization metric

In [ ]:
from ibm_watson_machine_learning.experiment import AutoAI
from ibm_watson_machine_learning.utils.autoai.enums import PredictionType, Metrics, SamplingTypes
from ibm_watson_machine_learning.utils.autoai.enums import ClassificationAlgorithms as CA
from ibm_watson_machine_learning.utils.autoai.enums import BatchedClassificationAlgorithms as BCA

experiment = AutoAI(wml_credentials, project_id=project_id)

pipeline_optimizer = experiment.optimizer(
    name='Electricity Prediction - AutoAI',
    prediction_type=AutoAI.PredictionType.CLASSIFICATION,
    prediction_column='class',
    scoring=AutoAI.Metrics.ROC_AUC_SCORE,
    sample_size_limit=635897,
    sampling_type="first_n_records",
    retrain_on_holdout=True,
    include_only_estimators=[CA.EX_TREES, CA.LGBM, CA.RF, CA.SnapRF, CA.SnapBM, CA.XGB],
    include_batched_ensemble_estimators=[BCA.EX_TREES, BCA.LGBM, BCA.RF, BCA.SnapBM, BCA.SnapRF, BCA.XGB],
    use_flight=True,
    
)

Configuration parameters can be retrieved via `get_params()`.

In [ ]:
pipeline_optimizer.get_params()

<a id="run"></a>
## 4. Experiment run

Call the `fit()` method to trigger the AutoAI experiment. You can either use interactive mode (synchronous job) or background mode (asychronous job) by specifying `background_model=True`.

In [ ]:
run_details = pipeline_optimizer.fit(
            training_data_reference=training_data_reference,
            background_mode=True)

In [ ]:
run_details

You can use the `get_run_status()` method to monitor AutoAI jobs in background mode.

In [ ]:
pipeline_optimizer.get_run_status()

<a id="comparison"></a>
## 5. Pipelines comparison and testing

You can list trained pipelines and evaluation metrics information in
the form of a Pandas DataFrame by calling the `summary()` method. You can
use the DataFrame to compare all discovered pipelines and select the one
you like for further testing.

In [ ]:
summary = pipeline_optimizer.summary()
summary

You can visualize the scoring metric calculated on a holdout data set.

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"

summary.holdout_roc_auc.plot()

### Get selected pipeline model

Download and reconstruct a scikit-learn pipeline model object from the
AutoAI training job.

In [ ]:
pipeline_model = pipeline_optimizer.get_pipeline("Pipeline_9")

Check features importance for selected pipeline.

In [ ]:
pipeline_optimizer.get_pipeline_details("Pipeline_9")['features_importance']

### Convert the pipeline model to a Python script and download it

In [ ]:
from ibm_watson_machine_learning.helpers import pipeline_to_script
pipeline_to_script(best_pipeline)

### Visualize pipeline

In [ ]:
pipeline_model.visualize()

Each node in the visualization is a machine-learning operator
(transformer or estimator). Each edge indicates data flow (transformed
output from one operator becomes input to the next).  The input to the
root nodes is the initial dataset and the output from the sink node
is the final prediction.  When you hover the mouse pointer over a
node, a tooltip shows you the configuration arguments of the
corresponding operator (tuned hyperparameters). When you click on the
hyperlink of a node, it brings you to a documentation page for the
operator.

### Pipeline source code

In [ ]:
pipeline_model.pretty_print(ipython_display=True, astype='sklearn')

In the pretty-printed code, `>>` is the pipe combinator (dataflow
edge) and `&` is the and combinator (combining multiple subpipelines).
They correspond to the `make_pipeline` and `make_union` functions from
scikit-learn, respectively. If you prefer the functions, you can
instead pretty-print your pipeline with
`best_pipeline.pretty_print(ipython_display=True, combinators=False)`.

### Reading training data from COS

In [ ]:
train_df = pipeline_optimizer.get_data_connections()[0].read()

train_X = train_df.drop(['class'], axis=1).values
train_y = train_df['class'].values

### Test pipeline model locally

In [ ]:
predicted_y = pipeline_model.predict(train_X)
predicted_y[:5]

<a id="incremental"></a>
## 6. Incremental learning of BatchedTreeEnsemble pipelines

In this section you learn to train AutoAI models incrementally.

### Data loader

Create DataLoader iterator to retrieve training dataset in batches (Pandas DataFrame). DataLoader is `Torch` compatible (`torch.utils.data`).

**Note**: If reading data results in an error, provide data as batches (Pandas DataFrame). It may be necessary to use methods for initial data pre-processing like: e.g. `DataFrame.dropna()`, `DataFrame.drop_duplicates()`, `DataFrame.sample()`.

In [ ]:
from ibm_watson_machine_learning.data_loaders import experiment as data_loaders
from ibm_watson_machine_learning.data_loaders.datasets import experiment as datasets

dataset = datasets.ExperimentIterableDataset(
    connection=training_data_reference[0],
    with_subsampling=False,
    experiment_metadata = {"prediction_type":"classification"},
    number_of_batch_rows=9062,
    _wml_client = client
)

data_loader = data_loaders.ExperimentDataLoader(dataset=dataset)

### Continue model training

In [ ]:
from sklearn.metrics import get_scorer

scorer = get_scorer(AutoAI.Metrics.ROC_AUC_SCORE)

In [ ]:
run_id = run_details['metadata']['id']

### Fit pipeline model in batches (`partial_fit`)
In this cell, the pipeline is incrementally fitted using data batches.


**Note**: You can also use custom code for data reading and model training. The `pipeline` supports incremental learning via `partial_fit` calls.

If you need, you can evaluate the pipeline using custom holdout data. Provide the `X_test`, `y_test` and call `scorer` on them.

In [ ]:
pipeline_model.steps[-1][-1].impl.max_sub_ensembles = 5

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

partial_fit_scores = []
f = go.FigureWidget()
f.layout.title.text = 'AutoAI incremental learning curve.'
f.add_scatter(y=partial_fit_scores, name='BatchedEnsemble')

In [ ]:
i = 0
iters = []

for batch_df in data_loader:
    X_train = batch_df.drop(['class'], axis=1).values
    y_train = batch_df['class'].values
    pipeline_model = pipeline_model.partial_fit(X_train, y_train, classes=pd.unique(train_y).tolist(), freeze_trained_prefix=True)
    pipeline_score = scorer(pipeline_model, X_train, y_train)
    partial_fit_scores.append(pipeline_score)
    f.data[0].y = partial_fit_scores
    print('Pipeline - batch: {0} | score: {1}'.format(iter, pipeline_score))

Test the fitted pipeline (`predict`).

In [ ]:
pipeline_model.predict(train_X)

<a id="cleanup"></a>
## 7. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 8. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `ibm-watson-machine-learning` to run AutoAI experiments. 
 
 Check out our _[Online Documentation](https://www.ibm.com/cloud/watson-studio/autoai)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Szymon Kucharczyk**, Staff Software Engineer in Watson Machine Learning at IBM

Copyright © 2022 IBM. This notebook and its source code are released under the terms of the MIT License.